In [ ]:
from typedb.client import TypeDBClient, TypeDB, SessionType, TransactionType
from typedb.common.exception import TypeDBClientException
from typedb.common.label import Label
from schema_utils import *
from typedb.concept.type.type import _Type as tt
from typedb.concept.type.role_type import RoleType
from typedb.concept.type.relation_type import RelationType
from typedb.logic.rule import Rule
import json
from linkml_runtime.utils.schemaview import SchemaView
from linkml_runtime.linkml_model.meta import ClassDefinition

In [ ]:
yaml_file = "../schema/biolink_sdtm_v0.yaml"

view = SchemaView(yaml_file)
view.imports_closure()

# typedb schema output file 
# TypeDB Native types: https://docs.vaticle.com/docs/schema/overview
# thing, entity, attribute, relation, role
typeql_schema = "biolink_sdtm_schema.tql"

### Create new DB if Biolink DB does not exist

In [ ]:
db_name = 'biolink_db_test'
with TypeDB.core_client("localhost:1729") as client:
    if client.databases().contains(db_name):
        client.databases().get(db_name).delete()
    client.databases().create(db_name)
    


### Add types to schema

In [ ]:

## ------------
## TYPES
## ------------

# typedb only has following types: https://docs.vaticle.com/docs/schema/overview
# long, double, string, boolean, datetime
typedb_types = ['long', 'double', 'string', 'boolean', 'datetime']

# dictionary to map biolink types to typedb types
type_mapper = {"int":"double", "date":"datetime", "float":"long", "XSDDate":"datetime", "str":"string"}
         
bl_special_types = build_types_dict(view, typedb_types, type_mapper)
            

### Add slots to schema

In [ ]:
## ------------
## SLOTS
## ------------

bl_special_types = add_slots_to_dict(bl_special_types, view, typedb_types, type_mapper)


### Add entities

In [ ]:
## ------------------
## THINGS/ENTITIES
## ------------------

# uriorcurie / Uniform Resource Identifiers (URIs) or CURIE (or Compact URI) is a default type in linkml
# entity is the root Biolink Model class for all things and informational relationships, real or imagined.

typeql = convert_to_camelCase("named thing")+" sub entity;\n\n"

entities_dict = build_entities_dict(view)


### Roles/associations

In [ ]:


roles_dict, entities_dict = build_roles_dict(view, entities_dict)

print(f"Roles added: {len(roles_dict)}")
# parent_class = 'association'
# travel_descent(parent_class, roles_dict, view)
# roles_dict = {k: roles_dict[k] for k in ['association', 'gene to gene association', 'pairwise gene to gene interaction']}

### Write schema variables to JSON

In [ ]:

json_schema = { 
    'types': bl_special_types,
    'entities': entities_dict,
    'roles': roles_dict}


with open('bl_typedb_schema.json', 'w') as outfile:
    json.dump(json_schema, outfile, indent=4)      

### Commit schema to TypeDB database

In [ ]:
print("defining types")

for i in bl_special_types:
    if 'value' in bl_special_types[i].keys():
        query = build_tql_define(convert_to_snakeCase(i), bl_special_types[i]['type'], 
                                abstract=bl_special_types[i]['abstract'], 
                                value=bl_special_types[i]['value'])
    else:
        query = build_tql_define(convert_to_snakeCase(i), bl_special_types[i]['type'], 
                                abstract=bl_special_types[i]['abstract'])
    
    print(query)
    write_query_transaction(query, db_name)

print("defining associations")
for r in roles_dict:  
    query = build_tql_rel(roles_dict[r]['name'], 
                        roles_dict[r]['type'], 
                        roles_dict[r]['relates'], 
                        abstract=roles_dict[r]['abstract'])
    print(query)
    write_query_transaction(query, db_name)

print("defining roles")
for r in roles_dict:
    for r in roles_dict:  
        if 'plays' in roles_dict[r].keys():
            
            query = build_tql_rel(roles_dict[r]['name'], 
                                                        roles_dict[r]['type'], 
                                                        roles_dict[r]['relates'], 
                                                        plays = roles_dict[r]['plays'],
                                                        abstract=roles_dict[r]['abstract'])
            
            print(query)
            write_query_transaction(query, db_name)

print("defining entities")
for e in entities_dict:
    query = build_tql_entity(entities_dict[e]['name'], entities_dict[e]['type'], entities_dict[e]['attributes'])
    print(query)
    write_query_transaction(query, db_name)

            # write_transaction.commit()

### Dump TypeDB schema as `.tql` file 

In [ ]:
db_name = 'biolink_db9'
with TypeDB.core_client("localhost:1729") as client:
    with client.session(db_name, SessionType.DATA) as session:
        a = session.database().schema()
    with open("biolink_typedb_schema.tql", "w") as f:
        f.write(a)